# usual imports


In [12]:

import os
import sys
import json
import folium
import pandas as pd
import branca.colormap as cm # for colormap work...not called yet
import webbrowser
import jenkspy



# lets see what versions we are working with...folium is dropping 2.7 support soon


In [13]:

print("Python version is %s.%s.%s" % sys.version_info[:3])
print("folium version is" + " " + folium.__version__)


Python version is 3.7.3
folium version is 0.8.3



# bring in the data and geographies


In [14]:

tables_src = 'ACS_17_5YR_S0101_DependencyRatios_NoDescription.csv'
geo_src = 'CENTRAL_SANDS_TRACTS.json'
table = pd.read_csv(tables_src)
tracts = json.load(open(geo_src))



# lets see what the coloumn of interest looks like


In [15]:

ratio_dict = table.set_index('GEOID')['HC01_EST_VC40'].sort_values(ascending=True)
print(ratio_dict)


GEOID
1400000US55097960400      9.6
1400000US55073000100      9.9
1400000US55097961000     10.4
1400000US55073000602     14.6
1400000US55097960900     14.9
1400000US55097960300     16.4
1400000US55073000200     17.8
1400000US55141010300     20.2
1400000US55073001201     20.2
1400000US55115101100     21.5
1400000US55097960702     21.5
1400000US55137960700     21.6
1400000US55073001600     22.5
1400000US55073001102     22.6
1400000US55073001104     23.0
1400000US55073001103     23.1
1400000US55097961200     23.4
1400000US55135100800     23.5
1400000US55141010100     23.7
1400000US55097960701     23.8
1400000US55073000500     24.5
1400000US55073002300     24.9
1400000US55073002000     25.2
1400000US55097961300     25.8
1400000US55073001202     25.9
1400000US55141010700     26.0
1400000US55073001900     26.1
1400000US55073002100     26.3
1400000US55073000601     26.3
1400000US55073001500     26.3
                        ...  
1400000US55135101100     34.1
1400000US55115100600     35.0
1400


# use jenkspy to get our class break points


In [16]:

datasrc = table['HC01_EST_VC40'].dropna(how='any').sort_values(ascending=True)
breaks = jenkspy.jenks_breaks(datasrc, nb_class=8)
print(breaks)


[9.6, 17.8, 27.5, 34.1, 42.9, 50.2, 65.4, 89.5, 106.8]



# create our folium map


In [17]:

m = folium.Map(
    tiles='CartoDB positron',
    zoom_start=8.5,
    control_scale=True,
    attr='CSR Productions'
)


In [18]:
m


# fix the position on opening to Central Wisconsin tracts BB


In [19]:

m.fit_bounds([[45.120608,-90.318032],[43.641046,-88.605033]])



# create the choropleth by joining our data coloumn to geography


In [20]:

folium.Choropleth(
    geo_data=tracts,
    data=table,
    name='Where dem old folks at?',
    columns=['GEOID2', 'HC01_EST_VC40'],
    key_on='feature.properties.GEO_Join',
    fill_color='YlOrRd',
    fill_opacity=0.70,
    bins=[9.6, 17.8, 27.5, 34.1, 42.9, 50.2, 65.4, 89.5, 106.8],
    highlight=True,
    legend_name='Old Age Dependency Ratio, 2013-2017',
    show=False
).add_to(m)



# add our control


In [21]:

folium.LayerControl().add_to(m)



# open in notebook


In [23]:

m
